In [1]:
import tensorflow as tf

import numpy as np
import os
import time

2023-05-04 16:47:08.457254: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-04 16:47:08.837925: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-04 16:47:08.837950: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-04 16:47:10.234649: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
path_to_file = 'evgenyi_onegin.txt'

In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 286984 characters


In [4]:
print(text[2000:2500])

          Так думал молодой повеса,
                        Летя в пыли на почтовых,
                        Всевышней волею Зевеса
                        Наследник всех своих родных.
                        Друзья Людмилы и Руслана!
                        С героем моего романа
                        Без предисловий, сей же час
                        Позвольте познакомить вас:
                        Онегин, добрый мой приятель,
                        Родился на брегах Невы,
               


In [5]:
text = text + text

In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

131 unique characters


In [7]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [8]:
text_as_int, text[:500], len(text_as_int), len(text)

(array([ 71, 110, 104, ..., 104, 121,   0]),
 'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                                Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Святой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высо',
 573968,
 573968)

### train and target

In [9]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

А
л
е
к
с


2023-05-04 16:47:23.836718: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-05-04 16:47:23.837016: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-04 16:47:23.837057: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mvkomarov-h): /proc/driver/nvidia/version does not exist
2023-05-04 16:47:23.866489: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '
'Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Свят'
'ой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких д'


In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first example input and target values:

In [12]:
for input_example, target_example in  dataset.take(2):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
Input data:  '      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                       '
Target data: '     Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '


In [13]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [14]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [15]:
# model = tf.keras.Sequential(
#     [
#         tf.keras.layers.Embedding(vocab_size, embedding_dim),
#         tf.keras.layers.LSTM(rnn_units, return_sequences=True),
#         tf.keras.layers.Dense(vocab_size)
#     ]
# )

In [16]:
# class RNNgenerator(tf.keras.Model):
#     def __init__(self, vocab_size, embedding_dim, batch_size):
#         super(RNNgenerator, self).__init__()
        
#         self.emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
                                 
#         self.gru1 = tf.keras.layers.GRU(rnn_units,
#                             return_sequences=True,
#                             recurrent_initializer='glorot_uniform')
#         self.gru2 = tf.keras.layers.GRU(rnn_units,
#                             return_sequences=True,
#                             recurrent_initializer='glorot_uniform')
                           
#         self.fc = tf.keras.layers.Dense(vocab_size)

#     def call(self, x):
#         emb_x = self.emb(x)
#         x1 = self.gru1(emb_x)
#         x = x1
#         for _ in range(3):
#             x = self.gru2(x)
#         #x = self.gru1(x)
#         x = (x + x1)/2
#         return self.fc(x)

# model = RNNgenerator(vocab_size, embedding_dim, BATCH_SIZE)

In [17]:
# def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
#     model = tf.keras.Sequential([
#         tf.keras.layers.Embedding(vocab_size, embedding_dim,
#                                   batch_input_shape=[batch_size, None]),
                                 
#         tf.keras.layers.LSTM(rnn_units,
#                             return_sequences=True,
#                             stateful=True,
#                             recurrent_initializer='glorot_uniform'),

#         tf.keras.layers.LSTM(rnn_units,
#                             return_sequences=True,
#                             stateful=True,
#                             recurrent_initializer='glorot_uniform'),

#          tf.keras.layers.LSTM(rnn_units,
#                             return_sequences=True,
#                             stateful=True,
#                             recurrent_initializer='glorot_uniform'),
        
#         tf.keras.layers.LSTM(rnn_units,
#                             return_sequences=True,
#                             stateful=True,
#                             recurrent_initializer='glorot_uniform'),
                                   
#         tf.keras.layers.Dense(vocab_size)
#     ])
#     return model

In [18]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [19]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [20]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 131) # (batch_size, sequence_length, vocab_size)


In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 256)         33536     
                                                                 
 gru (GRU)                   (None, None, 1024)        3938304   
                                                                 
 gru_1 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 gru_2 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 dense (Dense)               (None, None, 131)         134275    
                                                                 
Total params: 16,701,315
Trainable params: 16,701,315
Non-trainable params: 0
_________________________________________________________________


## Train the model

In [22]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 131)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.881549


In [23]:
model.compile(optimizer='adam', loss=loss, run_eagerly=True)

### Configure checkpoints

In [24]:
!rm -rf ./training_checkpoints

In [25]:
!ls ./training_checkpoints

ls: cannot access './training_checkpoints': No such file or directory


In [26]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*3,
    save_weights_only=True)

### Execute the training

In [27]:
EPOCHS = 15

In [28]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/15
88/88 [==============================] - 899s 10s/step - loss: 1.9128
Epoch 2/15
88/88 [==============================] - 896s 10s/step - loss: 1.3880
Epoch 3/15
88/88 [==============================] - 897s 10s/step - loss: 1.2622
Epoch 4/15
88/88 [==============================] - 896s 10s/step - loss: 1.1544
Epoch 5/15
88/88 [==============================] - 897s 10s/step - loss: 1.0869
Epoch 6/15
88/88 [==============================] - 897s 10s/step - loss: 0.9667
Epoch 7/15
88/88 [==============================] - 896s 10s/step - loss: 0.8650
Epoch 8/15
88/88 [==============================] - 896s 10s/step - loss: 0.7842
Epoch 9/15
88/88 [==============================] - 896s 10s/step - loss: 0.6606
Epoch 10/15
88/88 [==============================] - 898s 10s/step - loss: 0.5622
Epoch 11/15
88/88 [==============================] - 897s 10s/step - loss: 0.4678
Epoch 12/15
88/88 [==============================] - 896s 10s/step - loss: 0.3747
Epoch 13/15
88/88 [======

## Generate text

In [29]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_15'

In [30]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 256)         33536     
                                                                 
 gru_3 (GRU)                 (None, None, 1024)        3938304   
                                                                 
 gru_4 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 gru_5 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 dense_1 (Dense)             (None, None, 131)         134275    
                                                                 
Total params: 16,701,315
Trainable params: 16,701,315
Non-trainable params: 0
_________________________________________________________________


In [32]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [33]:
text_ = generate_text(model, start_string=u"С героем моего романа")
print(text_)

С героем моего романа
    Сх Рдлеря         падняей)
                    О    черов-    бощечадолоезе       чей.            не в  ноданеднылу;
  Онотат      Ихолзить,
          Взиша      кананемно,               К,
      наля,
  мчекарадей    миймнналоре
          надццоших    ду зи!     сь          нерVI1ачавемубраливафrcLI1ище,          лик и     Пам      н.  сьмум    ж   Ic}
    преказ         ды            XeIh3жебрахо          пем Чь,
     Ямевижи р
      р
      жадитьнеши   Дв празашедрегдвоемупою
 и,
 фрыше


In [34]:
len(text_)

521